In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Imports
import torch
cuda = torch.cuda.is_available()
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import sys
sys.path.append("../../semi-supervised")
sys.path.append("../../")

In [ ]:
#### pubmed

import importlib
import datautils
import datasets
importlib.reload(datautils)
importlib.reload(datasets)
from datautils import get_data_krylov

labelled, unlabelled, validation, test = get_data_krylov(batch_size=128,
                                                                    labels_per_class=977,
                                                                    validation_ratio=0.25,
                                                                    dataset_name='pubmed',
                                                                    norm_flag=False,
                                                                    gamma=1,
                                                                    dataset_path='data/',
                                                                    n_workers = 0)

alpha = 0.01 * (len(unlabelled) + len(labelled)) / len(labelled)
print(alpha)
def binary_cross_entropy(r, x):
    return -torch.sum(x * torch.log(r + 1e-8) + (1 - x) * torch.log(1 - r + 1e-8), dim=-1)


In [8]:
###model definition
import models
importlib.reload(models)
from models import AuxiliaryDeepGenerativeModelDropout


y_dim = 5
n_feature = 932
z_dim = 50
a_dim = 50
h_dim = [512, 256]


dropout_rate = 0.3

###model definition
from models import AuxiliaryDeepGenerativeModelDropout

model = AuxiliaryDeepGenerativeModelDropout([n_feature, y_dim, z_dim, a_dim, h_dim],dropout_rate=dropout_rate)
model


optimizer = torch.optim.Adam(model.parameters(), lr=2e-4, betas=(0.9, 0.999))

from itertools import cycle
from inference import SVI, DeterministicWarmup

# We will need to use warm-up in order to achieve good performance.
# Over 200 calls to SVI we change the autoencoder from
# deterministic to stochastic.
beta = DeterministicWarmup(n=200)


if cuda: model = model.cuda()
elbo = SVI(model, likelihood=binary_cross_entropy, beta=beta)

In [ ]:
from torch.autograd import Variable
import random
from itertools import cycle

#pLsq = graph_unlabelled_data.pLsq
#Y = graph_unlabelled_data.Y
#print(Y.shape)

#if cuda:
#  pLsq = pLsq.cuda(device=0)
#  Y = Y.cuda(device=0)

max_accuracy_valid = 0
max_accuracy_test = 0

for epoch in range(200):
    model.train()
    total_loss, accuracy = (0, 0)
    accuracy_ = 0
    xx = 0
    uu = 0
    mm = 0
    for (x, y), (u, yy) in zip(cycle(labelled), unlabelled):
        # Wrap in variables
        x, y, u = Variable(x), Variable(y), Variable(u)
        yy = Variable(yy)

        if cuda:
            # They need to be on the same device and be synchronized.
            x, y = x.cuda(device=0), y.cuda(device=0)
            u, yy = u.cuda(device=0), yy.cuda(device=0)



        vat_loss = VATLoss(xi=xi, eps=eps, ip=ip)
        cross_entropy = nn.CrossEntropyLoss()

        lds = vat_loss(model, u)
        output = model(x)
        classification_loss = cross_entropy(output, y)

        J_alpha = lds + alpha * classification_loss
        #from IPython.core.debugger import Pdb; Pdb().set_trace()

        J_alpha.backward()
        optimizer.step()
        #optimizer.zero_grad()

        total_loss += J_alpha.item()
        accuracy += torch.eq(torch.max(output, dim=1)[1], torch.max(y, dim=1)[1]).detach().cpu().float().sum()
        mm += x.shape[0]
        # Add auxiliary classification loss q(y|x)
        #logits_ = model.classify(u)
        #print(yy)
        #accuracy_ += torch.mean((torch.max(logits_, 1)[1].data == torch.max(yy, 1)[1].data).float())
        #print(torch.max(yy, 1)[1].data)

    if epoch % 1 == 0:

        print("Epoch: {}".format(epoch))
        print("[Train]\t\t J_a: {:.2f}, accuracy: {:.4f}".format(total_loss / mm, accuracy / mm))

        if accuracy / mm > 0.93:
          alpha = 0.01 * (len(unlabelled) + len(labelled)) / len(labelled)

        total_loss, accuracy_valid = (0, 0)
        m = len(validation)
        for x, y in validation:

            x, y = Variable(x), Variable(y)
            if cuda:
                x, y = x.cuda(device=0), y.cuda(device=0)
            outputs = model(x)
            #L = -elbo(x, y)
            #U = -elbo(x)

            outputs = model(x)
            accuracy_valid += torch.eq(torch.max(outputs, dim=1)[1], torch.max(y, dim=1)[1]).detach().cpu().float().mean() / m


        print("[Validation]\t accuracy: {:.4f}".format(accuracy_valid))


        total_loss, accuracy_test = (0, 0)
        for x, y in test:
            m = len(test)

            with torch.no_grad():
                x, y = Variable(x), Variable(y)
                if cuda:
                  x, y = x.cuda(device=0), y.cuda(device=0)
                outputs = model(x)

            accuracy_test += torch.eq(torch.max(outputs, dim=1)[1], torch.max(y, dim=1)[1]).detach().cpu().float().mean() / m

        if max_accuracy_valid < accuracy_valid:
            max_accuracy_valid = accuracy_valid
            max_accuracy_test = accuracy_test
        if max_accuracy_valid == accuracy_valid and max_accuracy_test < accuracy_test:
            max_accuracy_test = accuracy_test

        m = len(test)
        print("[Test]\t accuracy_current: {:.4f}, accuracy_max: {:.4f}".format(accuracy_test, max_accuracy_test))

